This file is for testing sar_dataset_loader.py

TODOS (All done):
1. ~~Read from real files or metadata dataframe~~
2. ~~Depth (height) adjust the wind speed~~
3. ~~Make sure the logic works in situations with both wave height parameters~~
4. ~~Make sure it works with files with both wave height and wind speed in the survey data~~

In [1]:
import sys
sys.path.insert(1, '/home/sarssw/filip/sarssw')

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import pickle
from tqdm import tqdm
import math
import pipeline.sar_dataset_loader as sar_dataset_loader
from importlib import reload

In [3]:
#Load bouy survey dataframe
bouy_survey_fn = '../bouy_survey/1h_survey/result_df'
swh_model_fn = '/data/exjobb/sarssw/model/2021_swh_era5_world_wide.nc'
wspd_model_fn = '/data/exjobb/sarssw/model/WIND_GLO_PHY_global/all.nc'

In [4]:
sar_dir = '/data/exjobb/sarssw/sar_dataset/'   
svc_file = '/home/sarssw/filip/sarssw/pipeline/homogenity_svc.pkl'
all_sar_images = os.listdir(sar_dir)

In [5]:
sar_paths = [os.path.join(sar_dir, f) for f in all_sar_images[:10]]

In [6]:
bouy_survey_path = '../bouy_survey/1h_survey/result_df'

with open(bouy_survey_path,'rb') as f_r:
    bouy_survey_df = pickle.load(f_r)

### 3. Make sure the logic works in situations with both wave height parameters

In [7]:
url = 'https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF.zip'
bouy = 'GL_TS_MO_6200078.nc'
sub_df = bouy_survey_df[(bouy_survey_df['bouy_file_name'] == bouy) & (bouy_survey_df['sar_url'] == url)]
display(sub_df)

,bouy_file_name,bouy_longitude,bouy_latitude,bouy_time,bouy_depth_index,bouy_depth,bouy_variable_name,bouy_variable_value,sar_url,sar_mode,sar_polarization,sar_platform,sar_start_time,sar_stop_time,sar_coordinates
38009,GL_TS_MO_6200078.nc,-2.787,47.238998,2021-04-09 18:00:00,0,0.0,VAVH,0.299,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-04-09 17:56:50,2021-04-09 17:57:15,"[[-3.24896, 48.195469], [-2.809912, 46.698505]..."
38238,GL_TS_MO_6200078.nc,-2.787,47.238998,2021-04-09 17:55:00,0,0.0,VHM0,0.399,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-04-09 17:56:50,2021-04-09 17:57:15,"[[-3.24896, 48.195469], [-2.809912, 46.698505]..."


In [8]:
sar_name = 'S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF'
reload(sar_dataset_loader)
sar_paths = [os.path.join(sar_dir, f) for f in all_sar_images if sar_name in f]
display(sar_paths)
#features = sar_dataset_loader.load_features_df(sar_paths, svc_file)
#display(features)
dataset_df = sar_dataset_loader.load_features_labels_df(sar_paths, svc_file, bouy_survey_fn, swh_model_fn, wspd_model_fn)
display(dataset_df)

['/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-2.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-5.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-0.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-8.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-3.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-4.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-6.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078

Calculating features


100%|██████████| 9/9 [00:02<00:00,  4.09it/s]


Loading wave height model
Loading wind speed model


100%|██████████| 1/1 [00:00<00:00, 21.65it/s]


file_name  \
sar_name                                           bouy_name                                                             
S1A_IW_GRDH_1SDV_20210409T175650_20210409T17571... GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   

                                                                    pol  \
sar_name                                           bouy_name              
S1A_IW_GRDH_1SDV_20210409T175650_20210409T17571... GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   

                                                                                                                  safe  \
sar_name                                     

## 4. Make sure it works with files with both wave height and wind speed in the survey data


In [3]:
v_count = bouy_survey_df.value_counts(subset=['bouy_file_name', 'sar_url'])
print(v_count.index[1])

reload(sar_dataset_loader)
sar_name = 'S1B_IW_GRDH_1SDV_20210104T064133_20210104T064158_025001_02F9B9_1DF4'

sar_paths = [os.path.join(sar_dir, f) for f in all_sar_images if sar_name in f]
#display(sar_paths)

dataset_df = sar_dataset_loader.load_features_labels_df(sar_paths, svc_file, bouy_survey_fn, swh_model_fn, wspd_model_fn)
display(dataset_df)

NameError: name 'bouy_survey_df' is not defined

## Test the train test val split

In [22]:
df_path = '/data/exjobb/sarssw/sar_dataset_features_labels_18_april/sar_dataset.pickle'

with open(df_path, 'rb') as f:
        fl_df = pickle.load(f)
        fl_df = fl_df.reset_index()
        
display(fl_df)

,sar_name,bouy_name,file_name,pol,safe,swath,platform,orbit_pass,product,platform_heading,...,SWH_value,SWH_source,SWH_lon,SWH_lat,SWH_time,WSPD_value,WSPD_source,WSPD_lon,WSPD_lat,WSPD_time
0,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,HH,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,EW,SENTINEL-1A,Descending,GRDH,-158.336231,...,2.740,bouy,-24.778168,65.69783,2021-01-03 08:00:00,9.247744,model,-24.778168,65.69783,2021-01-03 08:22:38
1,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,HV,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,EW,SENTINEL-1A,Descending,GRDH,-158.336231,...,2.740,bouy,-24.778168,65.69783,2021-01-03 08:00:00,9.247744,model,-24.778168,65.69783,2021-01-03 08:22:38
2,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,HH,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,EW,SENTINEL-1A,Descending,GRDH,-158.336231,...,2.740,bouy,-24.778168,65.69783,2021-01-03 08:00:00,9.247744,model,-24.778168,65.69783,2021-01-03 08:22:38
3,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,HV,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,EW,SENTINEL-1A,Descending,GRDH,-158.336231,...,2.740,bouy,-24.778168,65.69783,2021-01-03 08:00:00,9.247744,model,-24.778168,65.69783,2021-01-03 08:22:38
4,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,HH,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,EW,SENTINEL-1A,Descending,GRDH,-158.336231,...,2.740,bouy,-24.778168,65.69783,2021-01-03 08:00:00,9.247744,model,-24.778168,65.69783,2021-01-03 08:22:38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878397,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,GL_TS_MO_55038,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,HH,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,IW,SENTINEL-1B,Ascending,GRDH,-12.491007,...,0.729,bouy,151.500000,-23.90000,2021-12-17 08:30:00,6.269688,model,151.500000,-23.90000,2021-12-17 08:33:24
878398,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,GL_TS_MO_55038,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,HH,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,IW,SENTINEL-1B,Ascending,GRDH,-12.491007,...,0.729,bouy,151.500000,-23.90000,2021-12-17 08:30:00,6.269688,model,151.500000,-23.90000,2021-12-17 08:33:24
878399,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,GL_TS_MO_55038,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,HH,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,IW,SENTINEL-1B,Ascending,GRDH,-12.491007,...,0.729,bouy,151.500000,-23.90000,2021-12-17 08:30:00,6.269688,model,151.500000,-23.90000,2021-12-17 08:33:24
878400,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,GL_TS_MO_55038,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,HH,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,IW,SENTINEL-1B,Ascending,GRDH,-12.491007,...,0.729,bouy,151.500000,-23.90000,2021-12-17 08:30:00,6.269688,model,151.500000,-23.90000,2021-12-17 08:33:24


In [64]:
display(fl_df.iloc[0])
display(fl_df['file_name'].unique().shape)

file_name              S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
pol                                                                   HH
safe                   S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
swath                                                                 EW
platform                                                     SENTINEL-1A
orbit_pass                                                    Descending
product                                                             GRDH
platform_heading                                             -158.336231
polarisations                                                      HH HV
longitude                                                     -24.792572
latitude                                                       65.698284
time                                       2021-01-03T08:22:55.528182528
subimage_index                                                         3
hom_test                                           

(439417,)

In [29]:
" ".join(('abc', 'def'))

'abc def'

In [40]:
reload(sar_dataset_loader)

fl_df[['sar_name', 'bouy_name']].apply( sar_dataset_loader.hash_split, axis=1).value_counts(normalize=True)

train    0.602359
test     0.199935
val      0.197707
dtype: float64

In [46]:
[ser for ind,ser in fl_df[['sar_name', 'bouy_name']].iloc[0:10].iterrows()]

[sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
 bouy_name                                   AR_TS_MO_Blakksnes
 Name: 0, dtype: object,
 sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
 bouy_name                                   AR_TS_MO_Blakksnes
 Name: 1, dtype: object,
 sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
 bouy_name                                   AR_TS_MO_Blakksnes
 Name: 2, dtype: object,
 sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
 bouy_name                                   AR_TS_MO_Blakksnes
 Name: 3, dtype: object,
 sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
 bouy_name                                   AR_TS_MO_Blakksnes
 Name: 4, dtype: object,
 sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
 bouy_name                                   AR_TS_MO_Blakksnes
 Name: 5, dtype: object,
 sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
 bouy_name        

In [48]:
reload(sar_dataset_loader)

#get_random_value("ssssssss")
[(ser, sar_dataset_loader.hash_split(ser), sar_dataset_loader.hash_split(ser, seed_string="bop")) for ind,ser in fl_df[['sar_name', 'bouy_name']].iloc[0:10].iterrows()]

[(sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
  bouy_name                                   AR_TS_MO_Blakksnes
  Name: 0, dtype: object,
  'train',
  'test'),
 (sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
  bouy_name                                   AR_TS_MO_Blakksnes
  Name: 1, dtype: object,
  'train',
  'test'),
 (sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
  bouy_name                                   AR_TS_MO_Blakksnes
  Name: 2, dtype: object,
  'train',
  'test'),
 (sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
  bouy_name                                   AR_TS_MO_Blakksnes
  Name: 3, dtype: object,
  'train',
  'test'),
 (sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
  bouy_name                                   AR_TS_MO_Blakksnes
  Name: 4, dtype: object,
  'train',
  'test'),
 (sar_name     S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...
  bouy_name                                  

In [95]:
reload(sar_dataset_loader)

pd.Series([sar_dataset_loader.hash_split(fn, prob_list=[6, 2, 2], seed_string="a") for fn in fl_df['file_name'].unique()]).value_counts(normalize=True)

train    0.600737
val      0.200243
test     0.199021
dtype: float64

In [96]:
reload(sar_dataset_loader)

pd.Series([sar_dataset_loader.hash_split(fn, prob_list=[6, 2, 2], seed_string="") for fn in fl_df['file_name'].unique()]).value_counts(normalize=True)

train    0.600443
test     0.199908
val      0.199649
dtype: float64

## Test load_features_labels_df with split added


In [50]:
reload(sar_dataset_loader)

#Bouy survey and model paths
bouy_survey_fn = '../bouy_survey/1h_survey/result_df'
swh_model_fn = '/data/exjobb/sarssw/model/2021_swh_era5_world_wide.nc'
wspd_model_fn = '/data/exjobb/sarssw/model/WIND_GLO_PHY_global/all.nc'

#Sar image dir
sar_dir = '/data/exjobb/sarssw/sar_dataset/'   
all_sar_images = os.listdir(sar_dir)
all_sar_paths = [os.path.join(sar_dir, f) for f in all_sar_images[:1000]]

#Svc file for homogenity test
svc_file = './homogenity_svc.pkl'

dataset_df = sar_dataset_loader.load_features_labels_df(all_sar_paths, svc_file, bouy_survey_fn, swh_model_fn, wspd_model_fn)
display(dataset_df)

Calculating features


100%|██████████| 1000/1000 [00:27<00:00, 36.20it/s]


Loading wave height model
Loading wind speed model


100%|██████████| 991/991 [00:20<00:00, 49.27it/s]


,sar_name,bouy_name,file_name,pol,safe,swath,platform,orbit_pass,product,platform_heading,...,SWH_source,SWH_lon,SWH_lat,SWH_time,WSPD_value,WSPD_source,WSPD_lon,WSPD_lat,WSPD_time,split
0,S1A_EW_GRDH_1SDH_20210107T074925_20210107T0750...,AR_TS_MO_Hornafjardardufl,S1A_EW_GRDH_1SDH_20210107T074925_20210107T0750...,HH,S1A_EW_GRDH_1SDH_20210107T074925_20210107T0750...,EW,SENTINEL-1A,Descending,GRDH,-158.835944,...,bouy,-15.188500,64.196831,2021-01-07 07:53:19.000000256,5.890996,model,-15.188500,64.196831,2021-01-07 07:49:55.000,val
1,S1A_EW_GRDH_1SDH_20210107T074925_20210107T0750...,AR_TS_MO_Hornafjardardufl,S1A_EW_GRDH_1SDH_20210107T074925_20210107T0750...,HV,S1A_EW_GRDH_1SDH_20210107T074925_20210107T0750...,EW,SENTINEL-1A,Descending,GRDH,-158.835944,...,bouy,-15.188500,64.196831,2021-01-07 07:53:19.000000256,5.890996,model,-15.188500,64.196831,2021-01-07 07:49:55.000,val
2,S1A_EW_GRDH_1SDH_20210205T191535_20210205T1916...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210205T191535_20210205T1916...,HH,S1A_EW_GRDH_1SDH_20210205T191535_20210205T1916...,EW,SENTINEL-1A,Ascending,GRDH,-20.644168,...,bouy,-24.778168,65.697830,2021-02-05 19:00:00.000000000,13.222746,model,-24.778168,65.697830,2021-02-05 19:16:05.000,test
3,S1A_EW_GRDH_1SDH_20210205T191535_20210205T1916...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210205T191535_20210205T1916...,HV,S1A_EW_GRDH_1SDH_20210205T191535_20210205T1916...,EW,SENTINEL-1A,Ascending,GRDH,-20.644168,...,bouy,-24.778168,65.697830,2021-02-05 19:00:00.000000000,13.222746,model,-24.778168,65.697830,2021-02-05 19:16:05.000,test
4,S1A_EW_GRDH_1SDH_20210215T081417_20210215T0815...,AR_TS_MO_Flateyjardufl,S1A_EW_GRDH_1SDH_20210215T081417_20210215T0815...,HH,S1A_EW_GRDH_1SDH_20210215T081417_20210215T0815...,EW,SENTINEL-1A,Descending,GRDH,-159.285334,...,bouy,-23.110332,65.253166,2021-02-15 08:09:59.000000000,9.003396,model,-23.110332,65.253166,2021-02-15 08:14:47.000,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,S1B_IW_GRDH_1SDV_20211220T173301_20211220T1733...,NO_TS_MO_6200121,S1B_IW_GRDH_1SDV_20211220T173301_20211220T1733...,VH,S1B_IW_GRDH_1SDV_20211220T173301_20211220T1733...,IW,SENTINEL-1B,Ascending,GRDH,-15.589489,...,model,2.700000,53.500000,2021-12-20 17:33:13.500000000,2.100000,bouy,2.700000,53.500000,2021-12-20 18:00:00.000,train
1995,S1B_IW_GRDH_1SDV_20211222T060355_20211222T0604...,NO_TS_MO_Heimdal,S1B_IW_GRDH_1SDV_20211222T060355_20211222T0604...,VV,S1B_IW_GRDH_1SDV_20211222T060355_20211222T0604...,IW,SENTINEL-1B,Descending,GRDH,-162.129646,...,model,2.227300,59.574200,2021-12-22 06:04:07.500000000,4.200000,bouy,2.227300,59.574200,2021-12-22 06:00:00.000,train
1996,S1B_IW_GRDH_1SDV_20211222T060355_20211222T0604...,NO_TS_MO_Heimdal,S1B_IW_GRDH_1SDV_20211222T060355_20211222T0604...,VH,S1B_IW_GRDH_1SDV_20211222T060355_20211222T0604...,IW,SENTINEL-1B,Descending,GRDH,-162.129646,...,model,2.227300,59.574200,2021-12-22 06:04:07.500000000,4.200000,bouy,2.227300,59.574200,2021-12-22 06:00:00.000,train
1997,S1B_IW_GRDH_1SDV_20211222T171644_20211222T1717...,NO_TS_MO_Helgoland-NorthWR,S1B_IW_GRDH_1SDV_20211222T171644_20211222T1717...,VV,S1B_IW_GRDH_1SDV_20211222T171644_20211222T1717...,IW,SENTINEL-1B,Ascending,GRDH,-15.731676,...,bouy,7.818300,54.219299,2021-12-22 17:02:00.000000000,8.249531,model,7.818300,54.219299,2021-12-22 17:16:56.500,val


In [51]:
dataset_df['split'].value_counts(normalize=True)

train    0.606303
val      0.197599
test     0.196098
Name: split, dtype: float64

## Save sar_dataset dataframe with split, since i dont want to rerun the whole extraction ~12hours

In [53]:
reload(sar_dataset_loader)
df_path = '/data/exjobb/sarssw/sar_dataset_features_labels_18_april/sar_dataset.pickle'

with open(df_path, 'rb') as f:
        fl_df = pickle.load(f)
        
fl_df = fl_df.reset_index()
fl_df['split'] = fl_df[['sar_name', 'bouy_name']].apply(sar_dataset_loader.hash_split, axis=1)
        
display(fl_df)

,sar_name,bouy_name,file_name,pol,safe,swath,platform,orbit_pass,product,platform_heading,...,SWH_source,SWH_lon,SWH_lat,SWH_time,WSPD_value,WSPD_source,WSPD_lon,WSPD_lat,WSPD_time,split
0,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,HH,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,EW,SENTINEL-1A,Descending,GRDH,-158.336231,...,bouy,-24.778168,65.69783,2021-01-03 08:00:00,9.247744,model,-24.778168,65.69783,2021-01-03 08:22:38,train
1,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,HV,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,EW,SENTINEL-1A,Descending,GRDH,-158.336231,...,bouy,-24.778168,65.69783,2021-01-03 08:00:00,9.247744,model,-24.778168,65.69783,2021-01-03 08:22:38,train
2,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,HH,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,EW,SENTINEL-1A,Descending,GRDH,-158.336231,...,bouy,-24.778168,65.69783,2021-01-03 08:00:00,9.247744,model,-24.778168,65.69783,2021-01-03 08:22:38,train
3,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,HV,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,EW,SENTINEL-1A,Descending,GRDH,-158.336231,...,bouy,-24.778168,65.69783,2021-01-03 08:00:00,9.247744,model,-24.778168,65.69783,2021-01-03 08:22:38,train
4,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,AR_TS_MO_Blakksnes,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,HH,S1A_EW_GRDH_1SDH_20210103T082208_20210103T0823...,EW,SENTINEL-1A,Descending,GRDH,-158.336231,...,bouy,-24.778168,65.69783,2021-01-03 08:00:00,9.247744,model,-24.778168,65.69783,2021-01-03 08:22:38,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878397,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,GL_TS_MO_55038,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,HH,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,IW,SENTINEL-1B,Ascending,GRDH,-12.491007,...,bouy,151.500000,-23.90000,2021-12-17 08:30:00,6.269688,model,151.500000,-23.90000,2021-12-17 08:33:24,test
878398,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,GL_TS_MO_55038,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,HH,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,IW,SENTINEL-1B,Ascending,GRDH,-12.491007,...,bouy,151.500000,-23.90000,2021-12-17 08:30:00,6.269688,model,151.500000,-23.90000,2021-12-17 08:33:24,test
878399,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,GL_TS_MO_55038,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,HH,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,IW,SENTINEL-1B,Ascending,GRDH,-12.491007,...,bouy,151.500000,-23.90000,2021-12-17 08:30:00,6.269688,model,151.500000,-23.90000,2021-12-17 08:33:24,test
878400,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,GL_TS_MO_55038,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,HH,S1B_IW_GRDH_1SSH_20211217T083311_20211217T0833...,IW,SENTINEL-1B,Ascending,GRDH,-12.491007,...,bouy,151.500000,-23.90000,2021-12-17 08:30:00,6.269688,model,151.500000,-23.90000,2021-12-17 08:33:24,test


In [54]:
fl_df['split'].value_counts(normalize=True)

train    0.602359
test     0.199935
val      0.197707
Name: split, dtype: float64

In [55]:
df_save_path = '/data/exjobb/sarssw/sar_dataset_features_labels_18_april/sar_dataset_split.pickle'

#Save dataframe with pickle
fl_df.to_pickle(df_save_path)